In [2]:
def array_check(var):
    """function to check if the input has an array index associated with it"""
    if var != None:
        if '[' in var:                          # check for brackets
            arr = var[:-1].split('[')           # split the string and remove the ending bracket            
            return arr                          # return the list [variableName, arraySize]
        else:
            return var                          # else, return the non-array variable

In [3]:
def code_gen(Q):
    global ST
    """Code Generator printing MASM Assembly Code
    
        input: Quadruple Q = [x, y, op, z] 
                representing Three-address code:
                            x = y op z
        output: string of target code in MASM
    """
    
    # dictionary for what kind of conditional jump we need
    jump_dict = {'<': 'JL', '>': 'JG', '<=': 'JLE', '>=': 'JGE', '==': 'JE', '!=': 'JNE'}
    
    x = array_check(Q[0])
    y = array_check(Q[1])
    op = array_check(Q[2])
    z = array_check(Q[3])
    
    target = ""
    
    if op=="+":                            # Addition
        if type(y) != list: target += "MOV "+"R8, "+str(y)+"\n"
        else:
            target += "LEA RSI, "+str(y[0])+'\n'             # move the start of the array into rsi
            target += "MOV RDX, "+str(y[1])+'\n'             # move the index into rdx register
            target += "MOV R8, QWORD PTR [rsi+rdx*8]\n"
            
        if type(z) != list: target += "MOV "+"R9, "+str(z)+"\n"
        else:
            target += "LEA RSI, "+str(z[0])+'\n'             # move the start of the array into rsi
            target += "MOV RDX, "+str(z[1])+'\n'             # move the index into rdx register
            target += "MOV R9, QWORD PTR [rsi+rdx*8]\n"
            
        target += "ADD R8, R9\n"
        if type(x) != list: target += "MOV "+str(x)+", R8\n"
        else:
            target += "LEA RSI, "+str(x[0])+'\n'             # move the start of the array into rsi
            target += "MOV RDX, "+str(x[1])+'\n'             # move the index into rdx register
            target += "MOV QWORD PTR [rsi+rdx*8], R8\n"
    
    elif op=="-":                          # Subtraction x = y - z
        if type(y) != list: target += "MOV "+"R8, "+str(y)+"\n"
        else: 
            target += "LEA RSI, "+str(y[0])+'\n'             # move the start of the array into rsi
            target += "MOV RDX, "+str(y[1])+'\n'             # move the index into rdx register
            target += "MOV R8, QWORD PTR [rsi+rdx*8]\n"
            
        if type(z) != list: target += "MOV "+"R9, "+str(z)+"\n"
        else: 
            target += "LEA RSI, "+str(z[0])+'\n'             # move the start of the array into rsi
            target += "MOV RDX, "+str(z[1])+'\n'             # move the index into rdx register
            target += "MOV R9, QWORD PTR [rsi+rdx*8]\n"
            
        target += "SUB R8, R9\n"
        if type(x) != list: target += "MOV "+str(x)+", R8\n"
        else:
            target += "LEA RSI, "+str(x[0])+'\n'             # move the start of the array into rsi
            target += "MOV RDX, "+str(x[1])+'\n'             # move the index into rdx register
            target += "MOV QWORD PTR [rsi+rdx*8], R8\n"
    
    elif op=="*":                          # Multiplication x = y * z
        if type(y) != list: target += "MOV "+"rax, "+str(y)+"\n"
        else:
            target += "LEA RSI, "+str(y[0])+'\n'             # move the start of the array into rsi
            target += "MOV RDX, "+str(y[1])+'\n'             # move the index into rdx register
            target += "MOV rax, QWORD PTR [rsi+rdx*8]\n"
            
        if type(z) != list: target += "MOV "+"R8, "+str(z)+"\n"
        else:
            target += "LEA RSI, "+str(z[0])+'\n'             # move the start of the array into rsi
            target += "MOV RDX, "+str(z[1])+'\n'             # move the index into rdx register
            target += "MOV R8, QWORD PTR [rsi+rdx*8], rdx\n"
            
        target += "IMUL R8\n"
        if type(x) != list: target += "MOV "+str(x)+", rax\n"
        else:
            target += "LEA RSI, "+str(x[0])+'\n'             # move the start of the array into rsi
            target += "MOV RDX, "+str(x[1])+'\n'             # move the index into rdx register
            target += "MOV QWORD PTR [rsi+rdx*8], rax\n"
    
    elif op=="/":                         # Division x = y / z
        target += "xor rdx, rdx\n"
        target += "xor rax, rax\n"
        if type(y) != list: target += "MOV "+"rax, "+str(y)+"\n"
        else:
            target += "LEA RSI, "+str(y[0])+'\n'             # move the start of the array into rsi
            target += "MOV RDX, "+str(y[1])+'\n'             # move the index into rdx register
            target += "MOV rax, QWORD PTR [rsi+rdx*8]\n"
            
        if type(z) != list: target += "MOV "+"R8, "+str(z)+"\n"
        else:
            target += "LEA RSI, "+str(z[0])+'\n'             # move the start of the array into rsi
            target += "MOV RDX, "+str(z[1])+'\n'             # move the index into rdx register
            target += "MOV R8, QWORD PTR [rsi+rdx*8]\n"
            
        target += "IDIV R8\n"
        if type(x) != list: target += "MOV "+str(x)+", rax\n"
        else:
            target += "LEA RSI, "+str(x[0])+'\n'             # move the start of the array into rsi
            target += "MOV RDX, "+str(x[1])+'\n'             # move the index into rdx register
            target += "MOV QWORD PTR [rsi+rdx*8], rax\n"
    
    elif op=="%":                         # Mod x = y % z
        target += "xor rdx, rdx\n"
        target += "xor rax, rax\n"
        if type(y) != list: target += "MOV "+"rax, "+str(y)+"\n"
        else: 
            target += "LEA RSI, "+str(y[0])+'\n'             # move the start of the array into rsi
            target += "MOV RDX, "+str(y[1])+'\n'             # move the index into rdx register
            target += "MOV rax, QWORD PTR [rsi+rdx*8]\n"
            
        if type(z) != list: target += "MOV "+"R8, "+str(z)+"\n"
        else: 
            target += "LEA RSI, "+str(z[0])+'\n'             # move the start of the array into rsi
            target += "MOV RDX, "+str(z[1])+'\n'             # move the index into rdx register
            target += "MOV R8, QWORD PTR [rsi+rdx*8]\n"
            
        target += "IDIV R8\n"
        if type(x) != list: target += "MOV "+str(x)+", rdx\n"
        else:
            target += "LEA RSI, "+str(x[0])+'\n'             # move the start of the array into rsi
            target += "MOV RDX, "+str(x[1])+'\n'             # move the index into rdx register
            target += "MOV QWORD PTR [rsi+rdx*8], rdx\n"
    
    elif x == "label":#----------------Label Statement---------------------------
        target += '\n'+str(y)+':'+'\n'                       # y is a label - won't be an array
        
    elif x == 'JUMP':#----------------unconditional jump Statement---------------------------
        target += 'JMP '+str(y)+'\n'                         
        
    elif y in ['<', '>', '<=', '>=', '==', '!=']:#----------------conditional jump Statement---------------------------
        if type(x) != list and type(op) != list: 
            target += 'MOV R10, '+str(op)+'\n'           # move one to a register so we don't have two memory addresses
            target += 'CMP '+str(x)+', R10\n'    
        elif type(x) != list and type(op) == list: 
            target += "LEA RSI, "+str(op[0])+'\n'             
            target += "MOV RDX, "+str(op[1])+'\n' 
            target += "MOV R10, QWORD PTR [rsi+rdx*8]\n"
            target += 'CMP '+str(x)+', R10\n' 
        elif type(op) != list and type(x) == list: 
            target += "LEA RSI, "+str(x[0])+'\n'             
            target += "MOV RDX, "+str(x[1])+'\n' 
            target += "MOV R10, QWORD PTR [rsi+rdx*8]\n"
            target += 'CMP R10, '+str(op)+'\n' 
        else: 
            target += "LEA RSI, "+str(op[0])+'\n'             
            target += "MOV RDX, "+str(op[1])+'\n' 
            target += "MOV R10, QWORD PTR [rsi+rdx*8]\n"
            target += "LEA RSI, "+str(x[0])+'\n'             
            target += "MOV RDX, "+str(x[1])+'\n'
            target += "MOV R11, QWORD PTR [rsi+rdx*8]\n"
            target += 'CMP R11, R10\n' 
            
        target += str(jump_dict[str(y)])+' '+str(z)+'\n'     # y is a relop and z is a label - not arrays
    
    elif op == None:
        #print(x,"=",y,"","")#---------Assignment Statement----------------------
        if type(y) != str:                                      # constant or list on right
            if type(y) != list and type(x) != list:             # no lists              
                target += "MOV "+str(x)+", "+str(y)+"\n" 
            elif type(x) == list and type(y) != list:
                target += "LEA RSI, "+str(x[0])+'\n'             # move the start of the array into rsi
                target += "MOV RDX, "+str(x[1])+'\n'             # move the index into rdx register
                target += "MOV QWORD PTR [rsi+rdx*8], "+str(y)+"\n"      # move the value into the list
            elif type(y) == list and type(x) != list:
                target += "LEA RSI, "+str(y[0])+'\n'             # move the start of the array into rsi
                target += "MOV RDX, "+str(y[1])+'\n'             # move the index into rdx register
                target += "MOV R8, QWORD PTR [rsi+rdx*8]\n"      # move the value at the index into R8
                target += "MOV "+str(x)+", R8\n" 
            else:
                target += "LEA RSI, "+str(y[0])+'\n'             # move the start of the array into rsi
                target += "MOV RDX, "+str(y[1])+'\n'             # move the index into rdx register
                target += "MOV R8, QWORD PTR [rsi+rdx*8]\n"      # move the value at the index into R8
                
                target += "LEA RSI, "+str(x[0])+'\n'             # move the start of the array into rsi
                target += "MOV RDX, "+str(x[1])+'\n'             # move the index into rdx register
                target += "MOV QWORD PTR [rsi+rdx*8], R8\n"      # move the value into the list
                
        else:
            if type(y) != list: target += "MOV R8, "+str(y)+"\n"                  # variable or list on right
            else:
                target += "LEA RSI, "+str(y[0])+'\n'             # move the start of the array into rsi
                target += "MOV RDX, "+str(y[1])+'\n'             # move the index into rdx register
                target += "MOV R8, QWORD PTR [rsi+rdx*8]\n"      # move the value at the index into R8  
                
            if type(x) != list: target += "MOV "+str(x)+", R8\n" 
            else: 
                target += "LEA RSI, "+str(x[0])+'\n'             # move the start of the array into rsi
                target += "MOV RDX, "+str(x[1])+'\n'             # move the index into rdx register
                target += "MOV QWORD PTR [rsi+rdx*8], R8\n"      # move the value into the list
    
    return target